## Import Libs for Market Basket

In [148]:
#import basic Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Market Basket Models
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline

In [149]:
filenames = ['aisles','departments' ,'order_products__prior','order_products__train', 'orders','products']

In [150]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/'+ filenames[0] + '.csv')
departments = pd.read_csv('./data/'+ filenames[1] + '.csv')
order_products__prior = pd.read_csv('./data/'+ filenames[2] + '.csv')
order_products__train = pd.read_csv('./data/'+ filenames[3] + '.csv')
orders = pd.read_csv('./data/'+ filenames[4] + '.csv')
products = pd.read_csv('./data/'+ filenames[5] + '.csv')

## EDA

In [151]:
#checking the shape of each of the file and checks for any null values.
for name in filenames:
    print(name, eval(name).shape,": Null values?", eval(name).isnull().any()[0])

aisles (134, 2) : Null values? False
departments (21, 2) : Null values? False
order_products__prior (32434489, 4) : Null values? False
order_products__train (1384617, 4) : Null values? False
orders (3421083, 7) : Null values? False
products (49688, 4) : Null values? False


In [152]:
aisles.head()
departments.head()
#combine aisle and departments
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [155]:
opt = order_products__train.merge(products,how='left', on='product_id')
opt = opt.merge(departments,how='left', on='department_id')
opt = opt.merge(aisles,how='left', on='aisle_id')

In [156]:
#converting the id to string to preserve order.
opt['aisle_id']= opt['aisle_id'].astype(str)

In [157]:
#checking the top product aisles.  
opt[['aisle','aisle_id']]['aisle'].value_counts().head()

fresh vegetables              150609
fresh fruits                  150473
packaged vegetables fruits     78493
yogurt                         55240
packaged cheese                41699
Name: aisle, dtype: int64

In [158]:
#checking products by name. 
opt[['product_name','product_id']]['product_name'].value_counts().head()

Banana                    18726
Bag of Organic Bananas    15480
Organic Strawberries      10894
Organic Baby Spinach       9784
Large Lemon                8135
Name: product_name, dtype: int64

 ## Preprocessing for Market Basket

In [159]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [160]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

# of unique aisles = 134


In [161]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [162]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [163]:
opt[opt['aisle_id'] == 134]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle


In [164]:
range(len(opt['aisle_id'].unique()))

range(0, 134)

In [165]:
opt[opt['aisle_id'] == '1']['aisle'].value_counts()

prepared soups salads    2936
Name: aisle, dtype: int64

In [166]:
#basket.drop(columns=0,inplace=True)
#basket.rename(columns=range(len(opt['aisle_id'].unique())).tolist())
#basket[0].sum()
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [167]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [168]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [169]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))
#aisle_dict = dict(zip((opt['aisle_id'].unique()),opt['aisle'].unique()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  if __name__ == '__main__':


In [264]:
#aisle_dict
#len(basket_sets.columns)

In [171]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [172]:
basket_sets.head()

,prepared soups salads,specialty cheeses,energy granola bars,instant foods,marinades meat preparation,other,packaged meat,bakery desserts,pasta sauce,kitchen supplies,...,trail mix snack mix,feminine care,body lotions soap,tortillas flat bread,frozen appetizers sides,hot cereal pancake mixes,dry pasta,beauty,muscles joints pain relief,specialty wines champagnes
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [173]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

## Market Basket Results

In [174]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [175]:
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(refrigerated),(fresh fruits),0.133375,0.550099,0.086701,0.650057,1.181708,0.013332,1.285640
1,(fresh fruits),(refrigerated),0.550099,0.133375,0.086701,0.157610,1.181708,0.013332,1.028770
2,(packaged vegetables fruits),(fresh vegetables),0.383030,0.450975,0.252361,0.658853,1.460953,0.079624,1.609352
3,(fresh vegetables),(packaged vegetables fruits),0.450975,0.383030,0.252361,0.559589,1.460953,0.079624,1.400896
4,(frozen produce),(fresh fruits),0.128299,0.550099,0.093362,0.727694,1.322841,0.022785,1.652187


In the above example, the {Diaper} is the **antecedent** and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [176]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
27,"(eggs, fresh fruits)",(fresh vegetables),0.104025,0.450975,0.074355,0.714778,1.584960,0.027442,1.924901
28,"(fresh vegetables, fresh fruits)",(eggs),0.327333,0.147459,0.074355,0.227153,1.540445,0.026086,1.103117
29,(eggs),"(fresh vegetables, fresh fruits)",0.147459,0.327333,0.074355,0.504238,1.540445,0.026086,1.356836
30,(fresh vegetables),"(eggs, fresh fruits)",0.450975,0.104025,0.074355,0.164875,1.584960,0.027442,1.072864
34,(milk),(yogurt),0.223750,0.253405,0.086991,0.388787,1.534251,0.030292,1.221497


## Market Basket for Products - Tasks
Need to filter out products that have been reordered to reduce the size of the product matrix. Once I have the reordered products, I can join the one hot encoded items to the order Id's


In [266]:
reorders = opt[opt['reordered'] == 1]

In [267]:
reorders['product_id'] = reorders['product_id'].astype('int64')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [268]:
reorders.head()
reorders.shape

(828824, 9)

In [269]:
reorders.dtypes

order_id              int64
product_id            int64
add_to_cart_order     int64
reordered             int64
product_name         object
aisle_id             object
department_id         int64
department           object
aisle                object
dtype: object

In [270]:
# get list of hi volume products (products that occurr mo' than 1 tiempo)
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [271]:
# mask the reorders dataframe to only incluse dem hi vol prods, yo
reorders = reorders[reorders['product_id'].isin(hivol)]

In [272]:
#reorders['hi_dem'] = 
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [273]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [247]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [248]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [249]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [250]:
products.shape

(13087, 4823)

In [251]:
basket = hidem_ord[['order_id']].join(products)

In [252]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [253]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [254]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  if __name__ == '__main__':


In [255]:
len(pro_dict)
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [256]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [257]:
basket_sets.sum().head()

Organic 4% Milk Fat Whole Milk Cottage Cheese    1
Lightly Smoked Sardines in Olive Oil             0
Spring Water                                     2
Cage Free Extra Large Grade AA Eggs              0
Roasted Turkey                                   0
dtype: int64

In [258]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [259]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[['antecedants','consequents','confidence','lift']]

## Market Basket for Products

In [274]:
rules[(rules['lift'] >= 100) & (rules['confidence']>.5)]

,antecedants,consequents,confidence,lift
9,(White Peach),(Orange Mango Chicken),0.666667,100.106667
13,"(Chocolate Almond Crunch Ice Cream, Duck Eggs)",(Orange Mango Chicken),0.666667,100.106667
41,"(Chocolate Almond Crunch Ice Cream, White Beau...",(Orange Mango Chicken),0.666667,100.106667
56,(Whipping Cream Ultra Pasteurized),(Vanilla With Caramel Low Fat Ice Cream Cone),1.000000,220.823529
67,(Organic White English Muffins),(Chocolate Chip Cookie Dough Frozen Greek Yogu...,1.000000,268.142857
69,(Creamy Coconut & Peanut Spread),(Strawberry Rhubarb Yoghurt),0.666667,312.833333
84,(Organic Cashew Carrot Ginger Soup),(Berry Punch Flavored Fruit Drink),1.000000,170.636364
86,(Cauliettes Culinary Cuts),(Vanilla With Caramel Low Fat Ice Cream Cone),1.000000,220.823529
113,"(Chocolate Almond Crunch Ice Cream, Medium Roa...",(Cheese Shredded Mozzarella Reduced Fat 2%),0.666667,113.757576
120,"(Organic Creamies Strawberry, Raspberry & Carr...",(Organic Kiwi),1.000000,938.500000


In [275]:
#Export data for tableau analysis.
rules.to_csv('./data/prod_rules.csv')

In [216]:
hidem_ord = user_orders[user_orders['hi_dem'] == True]

In [217]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [230]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [231]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [232]:
basket = hidem_ord[['user_id']].join(products)

In [233]:
#return the total items in the basket from the aisles
basket = basket.groupby('user_id').sum()

In [234]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [235]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0007, use_colnames=True)

In [236]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[['antecedants','consequents','confidence','lift']]

In [280]:
rules.sort_values(['lift','confidence'],ascending=False).head()

,antecedants,consequents,confidence,lift
153,(Red Vine Tomato),(Organic Pink Lady (Cripps) Apple),1.000000,1251.333333
152,(Organic Pink Lady (Cripps) Apple),(Red Vine Tomato),0.666667,1251.333333
120,"(Organic Creamies Strawberry, Raspberry & Carr...",(Organic Kiwi),1.000000,938.500000
121,(Organic Kiwi),"(Organic Creamies Strawberry, Raspberry & Carr...",0.500000,938.500000
40,"(Chocolate Almond Crunch Ice Cream, Orange Man...",(White Beauty Bar Soap),0.500000,469.250000


In [ ]:
rules.to_csv('./data/user_rules.csv')

In [ ]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)